In [7]:
import pandas as pd
import numpy as np

# carga el archivo CSV en un DataFrame de Pandas
df = pd.read_csv('framingham.csv')

# elimina las filas que contienen valores "NA"
df.dropna(inplace=True)

# convierte las columnas de texto a valores numéricos
X = df.iloc[:, 5].values.astype(float).reshape(-1, 1)  # Pies cuadrados del espacio habitable interior
y = df.iloc[:, 2].values.astype(float).reshape(-1, 1)  # Precio

# agrega una columna de 1s a la matriz de características
Xr = np.hstack((np.ones_like(X), X))

print(Xr)
print(X.shape)
print(y.shape)


[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]
(3656, 1)
(3656, 1)


In [9]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures

# cargar los datos
data = np.genfromtxt('framingham.csv', delimiter=',', skip_header=1)

# extraer las variables independientes y dependientes
X = data[:, [0, 4, 7, 8, 9]]
y = data[:, 12]

# eliminar las filas con valores "NA"
X = X[~np.isnan(X).any(axis=1)]
y = y[~np.isnan(y)]

# generar características polinómicas
poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X)

n_samples = min(X_poly.shape[0], y.shape[0])
X_poly = X_poly[:n_samples]
y = y[:n_samples]

# ajustar el modelo logístico
logreg = LogisticRegression()
logreg.fit(X_poly, y)

# imprimir los coeficientes del modelo
print(logreg.coef_)

ValueError: Unknown label type: 'continuous'